In [ ]:
import sqlite3
from telegram import InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Updater, CommandHandler, CallbackQueryHandler

# Путь к базе данных
DB_PATH = 'fin_schedule.db'

# Функция для получения списка групп из базы данных
def get_groups():
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT DISTINCT group_name FROM schedule")
    groups = [row[0] for row in cursor.fetchall()]
    conn.close()
    return groups

# Функция для получения расписания из базы данных
def get_schedule(group_name, day_of_week):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    query = """
    SELECT time_period, schedule 
    FROM schedule 
    WHERE group_name = ? AND day_of_week = ?
    ORDER BY time_period
    """
    cursor.execute(query, (group_name, day_of_week))
    schedule = cursor.fetchall()
    conn.close()
    return schedule

# Стартовая команда
def start(update, context):
    groups = get_groups()
    keyboard = [[InlineKeyboardButton(group, callback_data=f"group_{group}")] for group in groups]
    reply_markup = InlineKeyboardMarkup(keyboard)
    update.message.reply_text("Выберите группу:", reply_markup=reply_markup)

# Обработка выбора группы
def select_group(update, context):
    query = update.callback_query
    query.answer()
    selected_group = query.data.split("_")[1]
    context.user_data['selected_group'] = selected_group
    
    days = ['Понедельник', 'Вторник', 'Среда', 'Четверг', 'Пятница']
    keyboard = [[InlineKeyboardButton(day, callback_data=f"day_{day}")] for day in days]
    keyboard.append([InlineKeyboardButton("Назад", callback_data="back_to_groups")])  # Кнопка "Назад"
    reply_markup = InlineKeyboardMarkup(keyboard)
    query.edit_message_text(f"Вы выбрали группу: {selected_group}\nТеперь выберите день недели:", reply_markup=reply_markup)

# Обработка выбора дня недели
def select_day(update, context):
    query = update.callback_query
    query.answer()
    selected_day = query.data.split("_")[1]
    selected_group = context.user_data['selected_group']
    
    # Получаем расписание
    schedule = get_schedule(selected_group, selected_day)
    
    if schedule:
        schedule_text = "\n\n".join([f"{time}: {lesson}" for time, lesson in schedule])  # Двойной отступ
    else:
        schedule_text = "На этот день расписание отсутствует."
    
    # Добавляем кнопку "Назад" для возврата к выбору дня недели
    keyboard = [[InlineKeyboardButton("Назад", callback_data=f"group_{selected_group}")]]
    reply_markup = InlineKeyboardMarkup(keyboard)
    query.edit_message_text(f"Расписание для группы {selected_group} на {selected_day}:\n\n{schedule_text}", reply_markup=reply_markup)

# Обработка кнопки "Назад" для возврата к выбору группы
def back_to_groups(update, context):
    query = update.callback_query
    query.answer()
    
    # Формируем меню выбора группы
    groups = get_groups()
    keyboard = [[InlineKeyboardButton(group, callback_data=f"group_{group}")] for group in groups]
    reply_markup = InlineKeyboardMarkup(keyboard)
    query.edit_message_text("Выберите группу:", reply_markup=reply_markup)

# Основная функция запуска бота
def main():
    TOKEN = "7993217613:AAHfB9kRBiroR3qVhUuN-2MRxOrUehdxPEU"

    # Создаем обновления (Updater)
    updater = Updater(TOKEN, use_context=True)

    # Обработчики команд
    dp = updater.dispatcher
    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(CallbackQueryHandler(select_group, pattern="^group_"))
    dp.add_handler(CallbackQueryHandler(select_day, pattern="^day_"))
    dp.add_handler(CallbackQueryHandler(back_to_groups, pattern="^back_to_groups"))

    # Запускаем бота
    updater.start_polling()
    updater.idle()

if __name__ == "__main__":
    main()
